![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # df_output processing, CSV file I/O (e.g. pd.read_csv)
import gc
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn import preprocessing
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input df_output files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [3]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# df_output

In [4]:
df_output_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(df_output_DIR+'/main_task.csv')
df_test = pd.read_csv(df_output_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(df_output_DIR+'/sample_submission.csv')


In [5]:
df_train.info()

In [6]:
df_train.head(5)
gc.collect()

In [7]:
df_test.info()
gc.collect()

In [8]:
df_test.head(5)

In [9]:
sample_submission.head(5)

In [10]:
sample_submission.info()

In [11]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

df_output = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

gc.collect()

In [12]:
df_output.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [13]:
df_output.sample(5)

In [14]:
df_output.Reviews[1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

# Cleaning and Prepping df_output
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/df_output-cleaning.png)

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

In [15]:
# Для примера я возьму столбец Number of Reviews
df_output['Number_of_Reviews_isNAN'] = pd.isna(df_output['Number of Reviews']).astype('uint8')

In [16]:
df_output['Number_of_Reviews_isNAN']

In [17]:
# Далее заполняем пропуски 0, вы можете попробовать заполнением средним или средним по городу и тд...
df_output['Number of Reviews'].fillna(df_output['Number of Reviews'].min(), inplace=True)


### 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [18]:
df_output.nunique(dropna=False)

Какие признаки можно считать категориальными?

Для кодирования категориальных признаков есть множество подходов:
* Label Encoding
* One-Hot Encoding
* Target Encoding
* Hashing

Выбор кодирования зависит от признака и выбраной модели.
Не будем сейчас сильно погружаться в эту тематику, давайте посмотрим лучше пример с One-Hot Encoding:
![](https://i.imgur.com/mtimFxh.png)

In [19]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
df_output = pd.get_dummies(df_output, columns=[ 'City',], dummy_na=True)

In [20]:
df_output.head(5)

gc.collect()

In [21]:
df_output.sample(5)

#### Возьмем следующий признак "Price Range".

In [22]:
df_output['Price Range'].value_counts()

По описанию 'Price Range' это - Цены в ресторане.  
Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами, например 1,2,3  
*Попробуйте сделать обработку этого признака уже самостоятельно!*

In [23]:
# Ваша обработка 'Price Range'

df_output['Price Range'].fillna((df_output['Price Range'].mode()[0]), inplace=True)
df_output['Price Range'].fillna(df_output['Price Range'].mode()[0], inplace=True)
val_dict = {'$': 1, '$$ - $$$': 2, '$$$$': 3}
df_output['Price Range'] = df_output['Price Range'].map(val_dict)
df_output['Price Range']

gc.collect()

> Для некоторых алгоритмов МЛ даже для не категориальных признаков можно применить One-Hot Encoding, и это может улучшить качество модели. Пробуйте разные подходы к кодированию признака - никто не знает заранее, что может взлететь.

### Обработать другие признаки вы должны самостоятельно!
Для обработки других признаков вам возможно придется даже написать свою функцию, а может даже и не одну, но в этом и есть ваша практика в этом модуле!     
Следуя подсказкам в модуле вы сможете более подробно узнать, как сделать эти приобразования.

In [24]:
# Функция для подсчета числа "кухонь"
def cuisine_count(clmn1, clmn2):
    clmn1=clmn1.str.replace('[','')
    clmn1=clmn1.str.replace(']','')
    
    clmn1=clmn1.str.replace("'", "")
    clmn1=clmn1.str.split(',')
    
    for i in clmn1.index:
        c=len(clmn1[i])
        clmn2.loc[i]=c
    return clmn1, clmn2
    


df_output['number of cuisines']='number of cuisines'

a='Cuisine'

df_output['Cuisine Style']=df_output['Cuisine Style'].fillna(a)

cuisine_count(df_output['Cuisine Style'],df_output['number of cuisines'])

c=np.mean(df_output['number of cuisines'])


#Эксперимент с новыми данными, но они не войдут, не делают модель лучше
df_output['Average of Cusines']=np.mean(df_output['number of cuisines'])
df_output['Cuisines on Average']=df_output['number of cuisines'].apply(lambda x: x/c)

#Выделяю время отзывов в отдельные столбцы
df_output['Reviews_time'] = df_output['Reviews']

df_output['Reviews_time'] = df_output['Reviews_time'].str.replace('[', '')
df_output['Reviews_time'] = df_output['Reviews_time'].str.replace(']', '')

df_output['Reviews_time'] = df_output['Reviews_time'].str.replace("'", "")
df_output['Reviews_time'] = df_output['Reviews_time'].str.split(',')

df_output['Time1'] = 0

df_output['Time2'] = 0
df_output['Timedelta'] = 'Timedelta'

df_output['Reviews_time'] = df_output['Reviews_time'].fillna(0)

for k in df_output['Reviews_time'].index:
    if df_output['Reviews_time'][k] != 0:

        df_output['Time1'].loc[k] = df_output['Reviews_time'][k][-1]
        df_output['Time2'].loc[k] = df_output['Reviews_time'][k][-2]
        if not '/201' in df_output['Reviews_time'].loc[k][-2]:
           
            df_output['Time2'].loc[k] = 0 #'00/00/0000'
           
    
df_output['Time1']=df_output['Time1'].str.replace(" ", "")
df_output['Time2']=df_output['Time2'].str.replace(" ", "")
df_output['Time1'] = df_output['Time1'].fillna(0)
#Перевожу в формат времени
df_output['Time1']=pd.to_datetime(df_output['Time1'])
df_output['Time2']=pd.to_datetime(df_output['Time2'])


#Новый столбец - разница в днях между последними отзывами
df_output['Timedelta']=df_output['Time2'].dt.day-df_output['Time1'].dt.day
df_output['Timedelta']=df_output['Timedelta'].fillna(0)

#Новый столбец - год отзыва
df_output['Year2']=df_output['Time2'].dt.year
df_output['Year2']=df_output['Year2'].fillna(0)
df_output['Year1']=df_output['Time1'].dt.year
df_output['Year1']=df_output['Year1'].fillna(0)
#Новый столбец - квартал отзыва
df_output['Time2']=df_output['Time2'].dt.quarter
df_output['Time2']=df_output['Time2'].fillna(4)
df_output['Time1']=df_output['Time1'].dt.quarter
df_output['Time1']=df_output['Time1'].fillna(4)


#Новый столбец - без отзывов
df_output['Number_of_Reviews_isNAN'] = pd.isna(df_output['Number of Reviews']).astype('uint8')
#df_output['Number_of_Reviews_isNAN']
# Нормализация Ranking
x_array = np.array(df_output['Ranking'])
normalized_arr = preprocessing.normalize([x_array])
print(normalized_arr)
df_output['Ranking_Norm']=normalized_arr.reshape(-1, 1)
# Эксперимент с новыми данными, неудачный
#df_output['Ranking_Reviews']=df_output['Ranking']/df_output['Number of Reviews']

display(df_output.head(5))

gc.collect()

![](https://cs10.pikabu.ru/post_img/2018/09/06/11/1536261023140110012.jpg)

# EDA 
[Exploratory df_output Analysis](https://ru.wikipedia.org/wiki/Разведочный_анализ_данных) - Анализ данных
На этом этапе мы строим графики, ищем закономерности, аномалии, выбросы или связи между признаками.
В общем цель этого этапа понять, что эти данные могут нам дать и как признаки могут быть взаимосвязаны между собой.
Понимание изначальных признаков позволит сгенерировать новые, более сильные и, тем самым, сделать нашу модель лучше.
![](https://miro.medium.com/max/2598/1*RXdMb7Uk6mGqWqPguHULaQ.png)

### Посмотрим распределение признака

In [25]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [26]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [27]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [28]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

>Подумайте как из этого можно сделать признак для вашей модели. Я покажу вам пример, как визуализация помогает находить взаимосвязи. А далее действуйте без подсказок =) 


### Посмотрим распределение целевой переменной

In [29]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [30]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [31]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [32]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(df_output.drop(['sample'], axis=1).corr(),)

Вообще благодаря визуализации в этом датасете можно узнать много интересных фактов, например:
* где больше Пицерий в Мадриде или Лондоне?
* в каком городе кухня ресторанов более разнообразна?

придумайте свои вопрос и найдите на него ответ в данных)

# df_output Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [33]:
del df_output
gc.collect()
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(df_output_DIR+'/main_task.csv')
df_test = pd.read_csv(df_output_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

df_output = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
df_output.info()

gc.collect()

In [34]:
def preproc_df_output(df_input):
    '''includes several functions to pre-process the predictor df_output.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['Restaurant_id','ID_TA'], axis = 1, inplace=True)
    #df_output.drop(['ID_TA'], axis = 1, inplace=True)
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...
    df_output['Number of Reviews'].fillna(df_output['Number of Reviews'].min(), inplace=True)
    # тут ваш код по обработке NAN
    # ....
    
    
    # ################### 3. Encoding ############################################################## 
    # для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
    df_output = pd.get_dummies(df_output, columns=[ 'City',], dummy_na=True)
    # тут ваш код не Encoding фитчей
    # ....
    #df_output['Price Range'].fillna((np.min(df_output['Price Range'])), inplace=True)
    df_output['Price Range'].fillna(df_output['Price Range'].mode()[0], inplace=True)
    val_dict = {'$': 1, '$$ - $$$': 2, '$$$$': 3}
    df_output['Price Range'] = df_output['Price Range'].map(val_dict)
    
    # ################### 4. Feature Engineering ####################################################
    # тут ваш код не генерацию новых фитчей
    # ....
    
    def cuisine_count(clmn1, clmn2):
        clmn1=clmn1.str.replace('[','')
        clmn1=clmn1.str.replace(']','')
    
        clmn1=clmn1.str.replace("'", "")
        clmn1=clmn1.str.split(',')
    
        for i in clmn1.index:
            c=len(clmn1[i])
            clmn2.loc[i]=c
        return clmn1, clmn2
    


    df_output['number of cuisines']='number of cuisines'

    a='Cuisine'

    df_output['Cuisine Style']=df_output['Cuisine Style'].fillna(a)

    cuisine_count(df_output['Cuisine Style'],df_output['number of cuisines'])
    
    gc.collect()
    
    c=np.mean(df_output['number of cuisines'])


    # Неудавшиеся эксперименты с данными
    #df_output['Average of Cusines']=np.mean(df_output['number of cuisines'])
    #df_output['Cuisines on Average']=df_output['number of cuisines'].apply(lambda x: x/c)
    #df_output['Ranking on Cusines']= df_output['Ranking'].apply(lambda x: x*df_output['Cuisines on Average'])
    
    #df_output['Restaurant_id']=df_output['Restaurant_id'].str.replace('id_','')

    #df_output['Restaurant_id']=df_output['Restaurant_id'].apply(lambda x: int(x))
    df_output['Reviews_time'] = df_output['Reviews']

    df_output['Reviews_time'] = df_output['Reviews_time'].str.replace('[', '')
    df_output['Reviews_time'] = df_output['Reviews_time'].str.replace(']', '')

    df_output['Reviews_time'] = df_output['Reviews_time'].str.replace("'", "")
    df_output['Reviews_time'] = df_output['Reviews_time'].str.split(',')

    df_output['Time1'] = 0

    df_output['Time2'] = 0
    df_output['Timedelta'] = 'Timedelta'

    df_output['Reviews_time'] = df_output['Reviews_time'].fillna(0)

    for k in df_output['Reviews_time'].index:
        if df_output['Reviews_time'][k] != 0:

            df_output['Time1'].loc[k] = df_output['Reviews_time'][k][-1]
            df_output['Time2'].loc[k] = df_output['Reviews_time'][k][-2]
            if not '/201' in df_output['Reviews_time'].loc[k][-2]:
           
                df_output['Time2'].loc[k] = 0 #'00/00/0000'
           
    
    df_output['Time1']=df_output['Time1'].str.replace(" ", "")
    df_output['Time2']=df_output['Time2'].str.replace(" ", "")
    df_output['Time1'] = df_output['Time1'].fillna(0)
    df_output['Time1']=pd.to_datetime(df_output['Time1'])
    df_output['Time2']=pd.to_datetime(df_output['Time2'])



    df_output['Timedelta']=df_output['Time2'].dt.day-df_output['Time1'].dt.day
    df_output['Timedelta']=df_output['Timedelta'].fillna(0)
    
    df_output['Year2']=df_output['Time2'].dt.year
    df_output['Year2']=df_output['Year2'].fillna(0)
    df_output['Year1']=df_output['Time1'].dt.year
    df_output['Year1']=df_output['Year1'].fillna(0)
    
    
    df_output['Time2']=df_output['Time2'].dt.quarter
    df_output['Time2']=df_output['Time2'].fillna(4)
    df_output['Time1']=df_output['Time1'].dt.quarter
    df_output['Time1']=df_output['Time1'].fillna(4)
    df_output.drop(['Timedelta'], axis = 1, inplace=True)
    
    df_output['Number_of_Reviews_isNAN'] = pd.isna(df_output['Number of Reviews']).astype('uint8')
    
    x_array = np.array(df_output['Ranking'])
    normalized_arr = preprocessing.normalize([x_array])
    print(normalized_arr)
    df_output['Ranking_Norm']=normalized_arr.reshape(-1, 1)
    
    #df_output['Ranking_Reviews']=df_output['Ranking']/df_output['Number of Reviews']
    
    
    gc.collect()
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    gc.collect()
    return df_output

>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). 

#### Запускаем и проверяем что получилось
gc.collect()

In [35]:
df_preproc = preproc_df_output(df_output)
df_preproc.sample(10)

In [36]:
df_preproc.info()

In [37]:
# Теперь выделим тестовую часть
train_df_output = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_df_output = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_df_output.Rating.values            # наш таргет
X = train_df_output.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [38]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [39]:
# проверяем
test_df_output.shape, train_df_output.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [40]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [41]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [42]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [43]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [44]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(20).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [45]:
test_df_output.sample(10)

In [46]:
test_df_output = test_df_output.drop(['Rating'], axis=1)

In [47]:
sample_submission

In [48]:
predict_submission = model.predict(test_df_output)

In [49]:
predict_submission

In [50]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
